<a href="https://colab.research.google.com/github/Erenoa6621/web_novel_analysis/blob/master/NarouRead.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os,time,datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen,urljoin,urlparse

from google.colab import auth
from oauth2client.client import  GoogleCredentials
import gspread

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

BOOKNAME = "なろう小説"
try:
  book = gc.open(BOOKNAME)
except:
  book = gc.create(BOOKNAME)

In [ ]:
BASEURL = 'https://ncode.syosetu.com/'
NCODE = 'n8817he'
url = f"{BASEURL}{NCODE}"

html = urlopen(url)
soup = BeautifulSoup(html,'lxml')
novel_title = soup.find("p",class_="novel_title").get_text(strip=True)
author = soup.find("div",class_="novel_writername").get_text(strip=True)
print(novel_title)
s = len(soup.select("dl.novel_sublist2"))

try:
  sheet = book.worksheet(novel_title[:16])
except:
  sheet = book.add_worksheet(novel_title[:16],5+s,6) 

In [ ]:
sheet.update_cell(1,1,"タイトル")
sheet.update_cell(1,2,"作品名："+novel_title)
sheet.update_acell("A2","著者")
sheet.update_acell("B2",author)
sheet.update_acell("A3","NCODE")
sheet.update_acell("B3",NCODE)

In [ ]:
i = 4
sheet.append_row(["更新","サブタイトル","前書き","あとがき","本文","URL"])
file = "Load.txt"
for data in soup.select("dl.novel_sublist2"):
  subtitle = data.select_one("dd").get_text(strip=True)
  update = data.select_one("dt.long_update").get_text(strip=True)
  texturl = urljoin(url,data.a['href'])
  print(subtitle,update,texturl)

  textdata = urlopen(texturl)
  textsoup = BeautifulSoup(textdata,"lxml")
  text = textsoup.select_one("div#novel_honbun").get_text(strip=True)

  preface = ''
  for tp in textsoup.select("div#novel_p"):
    preface += tp.get_text(strip=True)
  afterword = ''
  for aw in textsoup.select("div#novel_a"):
    afterword += aw.get_text(strip=True)
  list =[update,subtitle,preface,afterword,text,texturl]
  i+=1
  for j,data in enumerate(list):
    sheet.update_cell(i,j+1,data) 
    
  fileobj = open(file, "w", encoding = "utf_8")
  fileobj.write(subtitle);
  fileobj.write(text);



 ## time.sleep(1)